<center>
<img src="https://raw.githubusercontent.com/dvgodoy/PyTorch101_ODSC_Europe2020/master/images/linear_dogs.jpg" width="800">

# Основы глубинного обучения: домашнее задание 4
</center>

**ФИО:**

**Забавный факт о себе:**

## Общая информация

__Дата выдачи:__ 09.12.2023

__Дедлайн:__ 23:59MSK 18.12.2023

В случае этой домашки мягкий дедлан совпадает с жёстким.

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 5.2 балла (3 основных балла и ещё 2.2 бонусных).

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

import re

In [2]:

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# Описание данных

Данные можно [скачать с гугл-диска.](https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing) Если вы делаете задание в колабе, то строчки ниже сами скачают вам данные.



In [3]:
import gdown

url = "https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/news_data/ria_news.tsv',
 '/content/news_data/vk_comments.tsv',
 '/content/news_data/vk_news.tsv']

В таблице `ria_news.tsv`  лежат данные о новостях, вышедших на сайте РИА-НОВОСТИ с 15 марта 2018 года по 31 декабря 2018 года.

- `href` - уникальный идентификатор новости (ссылка на неё)
- `date` - дата публикации новости
- `time` - время публикации новости
- `title` - заголовок новости
- `snippet` - краткое описание новости
- `text` - текст новости
- `category` - категория новости
- `keywords` - ключевые слова (подкатегории новости)
- `shows` - счётчик с числом просмотров новости на сайте (на момент парсинга)

In [4]:
df_ria = pd.read_csv('news_data/ria_news.tsv', sep='\t')
df_ria = df_ria[~df_ria.tags.isnull()]
print(df_ria.shape)
df_ria.head()

(201708, 9)


,href,date,time,title,snippet,text,category,tags,shows
0,/20181231/1548961410.html,2018-12-31,"31 декабря 2018, 23:52",Нетаньяху не собирается в отставку в случае пр...,Премьер-министр Израиля Биньямин Нетаньяху не ...,"МОСКВА, 31 дек - РИА Новости. Премьер-министр ...",В мире,"Биньямин Нетаньяху, Израиль, В мире",728.0
1,/20181231/1548961364.html,2018-12-31,"31 декабря 2018, 23:19",Макрон в новогоднем обращении затронул тему ре...,"Результат реформ не может быть мгновенным, зая...","ПАРИЖ, 31 дек – РИА Новости. Результат реформ ...",В мире,"Эммануэль Макрон, Франция, В мире",3086.0
2,/20181231/1548961337.html,2018-12-31,"31 декабря 2018, 23:12",Аарон Рэмзи проведет переговоры с пятью топ-кл...,"Полузащитник лондонского ""Арсенала"" Аарон Рэмз...","МОСКВА, 31 дек - РИА Новости. Полузащитник лон...",NaN,ФК Арсенал (Лондон),183.0
3,/20181231/1548961304.html,2018-12-31,"31 декабря 2018, 23:09",Гол Азмуна принес сборной Ирана победу над кат...,Футболисты сборной Ирана одержали победу над к...,"МОСКВА, 31 дек - РИА Новости. Футболисты сборн...",NaN,"Сердар Азмун, Сборная Ирана по футболу",78.0
4,/20181231/1548961265.html,2018-12-31,"31 декабря 2018, 23:07",Пятая ракетка мира дель Потро пропустит Открыт...,Аргентинский теннисист Хуан Мартин дель Потро ...,"МОСКВА, 31 дек - РИА Новости. Аргентинский тен...",NaN,Теннис,79.0


Многие новостные агенства поддерживают странички в социальных сетях. Они постят туда самые сочные сюжеты. В таблице `vk_news.tsv` лежат данные о новостях, которые РИА запостили ВКонтакте в период времени с  `2017-09-29 01:28:55` по `2019-02-01 23:13:17`.

- `id` - уникальный идентификатор поста
- `href` - ссылка на сайт (если она была указана в посте)
- `datetime` - дата и время публикации новости
- `title` - заголовок новости
- `text` - текст новости в социальной сети
- `likes` - число лайков под постом
- `comments` - число комментариев под постом

In [5]:
df_vk = pd.read_csv('news_data/vk_news.tsv', sep='\t')
df_vk['snippet'] = df_vk['text']
df_vk.drop('text', axis=1, inplace=True)
print(df_vk.shape)
df_vk.head()

(19928, 7)


,id,href,datetime,title,likes,comments,snippet
0,24006362,/20190201/1550280358.html,2019-02-01 23:13:17,"В ДНР заявили о задержании диверсантов, причас...",15,28,NaN
1,24006240,/20190201/1550268781.html,2019-02-01 22:38:41,"Житель Урала ""заминировал"" ТЦ из-за снятия со...",32,42,NaN
2,24006100,/20190201/1550282212.html,2019-02-01 21:58:52,"В Черном море нашли ""потерянный флот Гитлера""",84,23,NaN
3,24005972,/20190202/1550283179.html,2019-02-01 21:27:06,В США освободили задержанную российскую актрис...,58,35,NaN
4,24005764,/20190201/1550262848.html,2019-02-01 20:55:54,Толкнувший Скабееву депутат Рады заявил о гроз...,45,145,NaN


В таблице `vk_comments.tsv` лежат комментарии к новостям.

- `id` - уникальный идентификатор комментария
- `post_id` - идентификатор новости, под которой был оставлен комментарий
- `datetime` - дата и время, когда был оставлен комментарий
- `text` - текст комментария
- `likes` - число лайков под комментарием

In [6]:
df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')
df_comments = df_comments[~df_comments.text.isnull()]
print(df_comments.shape)
df_comments.head()

<ipython-input-6-9f937a17e540>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')


(2612629, 5)


,id,post_id,datetime,text,likes
0,24006366.0,24006362.0,2019-02-01 23:14:14,ЧВК Вагнера?,5.0
1,24006370.0,24006362.0,2019-02-01 23:15:23,"[id4710641|Евгений], выздоравливай.",3.0
2,24006371.0,24006362.0,2019-02-01 23:16:21,"[id442655034|Андрей], искренне желаю этого все...",4.0
3,24006374.0,24006362.0,2019-02-01 23:16:38,Опять про Украину новости?,1.0
4,24006375.0,24006362.0,2019-02-01 23:16:40,Че такое ДНР?,2.0


# А что надо сделать то?

В тетрадке вам предстоит сделать следующие шаги:

1. Вы обучите нейросеть предсказывать категорию новости
2. Вы построите предсказания для тех новостей, где мы ничего не знаем о категории
3. Вы используете уже обученный для сентимент-анализа классификатор из библиотеки `hugging face` чтобы предсказать эмоциональную окраску каждого комментария
4. Вы проведёте аналитику по новостям, а именно построите топы из самых позитивных и негативных категорий и новостей

Для первого шага вам будет дан бэйзлайн. Если вы его прогоните, у вас получится базовая модель, которая даст некоторое качество решения задачи. Вам надо будет выяснить, насколько это качество оказалось хорошим, а затем внести в код некоторые улучшения.



## Часть 1: категоризация новостей (1.2 + 2 бонусных балла)

Каждой новости в соотвествие поставлены ключевые слова. Будем считать, что эти ключевые слова — тематики новости. Нужно научиться предсказывать тематики по тексту новости. Готовые тематики у нас есть только по новостям с сайта. Они за 2018 год. По новостям из ВКонтакте у нас тематик нет. Мы собираемся их предсказать.

Новости, опубликованные ВКонтакте, отличаются от новостей с сайта тем, что у них есть только титул и короткое описание. Странно будет обучать нейросеть на длинных текстах, а потом использовать её на коротких описаниях. Мы не будем так делать. Мы попробуем обучить базовый вариант нейронной сети только на заголовках новостей. Все, кто захочет получить бонусные баллы, смогут попробовать добавить в нейросеть сниппеты (так назыают короткие описания новостей).

## 1.1 Подготовка таргета

Поработаем с таргетом. Мы будем предсказывать переменную `tags`. Давайте выясним скоько уникальных тегов существует.

In [ ]:
from collections import Counter

# удалим все лишние пробелы и сделаем lowercase
df_ria['tags'] = (
    df_ria.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(df_ria.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))
tags_cnt.most_common()[-20:]

13344


[('блог анны завершинской об автоспорте - блоги', 1),
 ('министерство транспорта рб', 1),
 ('министерство здравоохранения грузии', 1),
 ('палех', 1),
 ('юрий посохов (хореограф)', 1),
 ('мария александрова', 1),
 ('том бенсон', 1),
 ('абдул каюм кочай', 1),
 ('нуман куртулмуш', 1),
 ('mipim', 1),
 ('владимир попов', 1),
 ('брюно женезио', 1),
 ('роберт фицо', 1),
 ('сергей пашинский', 1),
 ('валерия гонтарева', 1),
 ('нововоронеж', 1),
 ('императорское православное палестинское общество', 1),
 ('event_poslanie_prezidenta_rf_federalnomu_sobraniju', 1),
 ('фхтр', 1),
 ('игорь честин', 1)]

Всего в выборке есть порядка 13 000 тэгов. Многие встречаются всего по разу. Давайте оставим в выборке только те тэги, которые встречаются более 30 раз.

In [ ]:
target_tags = {tag for tag,cnt in tags_cnt.most_common() if cnt > 30}
len(target_tags)

1583

Закодируем теги для OHE.

In [ ]:
tag2idx = dict(zip(target_tags, range(len(target_tags))))
idx2tag = {jtem: item for item,jtem in tag2idx.items()}

Почистим таргет от лишних тэгов.

In [ ]:
df_ria['target_tags'] = (
    df_ria.tags.
    apply(lambda w: [tag2idx.get(item) for item in  w.split(',') if item in target_tags])
)

df_ria = df_ria[df_ria.target_tags.apply(len) > 0]
df_ria.shape

(201437, 10)

In [ ]:
df_ria.target_tags.values[:3]

array([list([538, 1362, 114]), list([1247, 201, 114]), list([1216])],
      dtype=object)

## 1.2 Подготовка текстов

Теперь займёмся предобработкой текстов. Приведём все слова к маленькому регистру и выбросим мусорные символы. В качестве токенов будем рассматривать отдельные слова.

Напомню, что мы пока что решили работать только с названиями статей. Поэтому вся предобработка применяется исключительно к ним. **Спойлер:** предобработку для сниппетов вы сделаете сами в первом же задании.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize

def normalise_text(text):
    if pd.isna(text):
      return "#UNKN"
    text = text.lower()

    # сурово регулярками выкидываем мусорные символы
    text = re.sub('[^а-яa-z0-9 ]', '', text)
    text = re.sub('pstrong', '', text)
    text = re.sub('strongp', '', text)
    text = re.sub('strong', '', text)
    return text.strip()



In [ ]:
df_ria['title_clean'] = df_ria.title.apply(normalise_text)

word_cnt = Counter(word_tokenize(' '.join(df_ria.title_clean.values)))
len(word_cnt)

112178

In [ ]:
word_cnt.most_common()[:10]

[('в', 127323),
 ('на', 44386),
 ('с', 26150),
 ('и', 21771),
 ('о', 19948),
 ('по', 17014),
 ('россии', 13494),
 ('не', 13483),
 ('сша', 9942),
 ('за', 9881)]

Давайте почистим словарь от стоп-слов и подготовим его к использованию внутри датасета. Мы будем с помощью словаря заменять слова на индексы. Добавим в словарь несколько специальных токенов для неизвестных слов и паддингов.

In [ ]:
from nltk.corpus import stopwords

stops_ru = set(stopwords.words('russian'))
len(stops_ru)

151

In [ ]:
vocabulary = {
    "#PAD#": 0, "#UNK#": 1
}

k = 2
for word, _ in word_cnt.most_common():
    if word not in stops_ru:
        vocabulary[word] = k
        k += 1

In [ ]:
len(vocabulary)

112030

Завернём код для создания словаря в функцию.

In [ ]:
def create_vocab(text, stops_ru=stops_ru):

    word_cnt = Counter(word_tokenize(text))
    vocabulary = {
        "#PAD#": 0, "#UNK#": 1
    }

    k = 2
    for word, _ in word_cnt.most_common():
        if word not in stops_ru:
            vocabulary[word] = k
            k += 1
    return vocabulary

__[0.2 балла] Задание 1:__

- Cделайте аналогичную предобработку титулов из таблички `df_vk`. Запишите получившийся результат в столбец `title_clean` по аналогии с таблицей `df_ria`.
- Сделайте для обеих таблиц предобработку колонок со сниппетами `snippet` и запишите получившийся результат в столбец `snippet_clean`. Все пропуски заполните токеном `"#UNKN"`.

In [ ]:
df_vk['title_clean'] = df_vk.title.apply(normalise_text)

df_vk['snippet_clean'] = df_vk.snippet.apply(normalise_text)

df_ria['snippet_clean'] = df_ria.snippet.apply(normalise_text)


## 1.3 Поставка данных

Пересечение сайта и ВК по опубликованным новостям довольно маленькое. Мы обучаем модель на данных с сайта. Предсказания мы будем строить на данных из ВК. У этих данных разная природа. В ВК описание статей и заголовки короче. Модель может хорошо показать себя на данных с новостного сайта, но сильно просесть в качестве на данных из ВК.

Давайте сохраним пересечение в отдельную табличку, чтобы на нём можно было понять, насколько сильно деградирует модель.

In [ ]:
ria_hrefs = set(df_ria.href.values)
vk_hrefs = set(df_vk.href.values)
test_hrefs = (vk_hrefs & ria_hrefs)

print('Размер отложенной выборки:', len(test_hrefs))

df = df_ria[~df_ria.href.isin(test_hrefs)]

Размер отложенной выборки: 1128


По странному совпадению (я правда не знаю почему) пересечение лежит в декабре. Мы будем его использовать как тестовую выборку.

In [ ]:
df_ria[df_ria.href.isin(test_hrefs)].date.min(), df_ria[df_ria.href.isin(test_hrefs)].date.max()

('2018-12-06', '2018-12-31')

Предположим, что мы делим выборку на обучающую и тестовую случайно. За один и тот же промежуток времени может выйти довольно большое число новостей с одинаковым заголовком. Давайте представим себе, что в тесте и трэйне есть много статей про одно и то же событие. Модель научилась на обучающей выборке хорошо его тегировать. Остальные события модель тегирует намного хуже. Метрики на тестовой выборке высокие. В следующем месяце СМИ перестают освещать это событие, в потоке новостей совершенно другие новости. Качество модели резко проседает.

Чтобы не напороться на завышенные метрики, обычно выборку дробят на обучающую и тестовую по времени. Тогда статьи из теста будут имитировать поток новых новостей, освещающих новые события.

In [ ]:
df.date.min(), df.date.max()

('2018-03-15', '2018-12-31')

__[0.2 балла] Задание 2:__ Разбейте выборку на обучающую, валидационную и тестовую. В тест возьмите весь декабрь. В валидацию октябрь и ноябрь.

In [ ]:
# your code here
# ♡ (´｡• ω •｡`)

df_test = df[df.date > '2018-11-99']
df_val = df[(df.date > '2018-09-99') & (df.date < '2018-11-99')]
df_train = df[df.date < '2018-09-99']

Сформируем отложенную выборку (пересечение ВКонтакте и РИА).

In [ ]:
df_oob = df_vk[df_vk.href.isin(test_hrefs)][['href', 'title_clean']]

df_ria_oob = df_ria[df_ria.href.isin(test_hrefs)][['href', 'target_tags']]
df_oob = df_oob.set_index('href').join(df_ria_oob.set_index('href')).reset_index()
df_oob.head()

,href,title_clean,target_tags
0,/20181231/1548960744.html,митрополит рязанский раскритиковал иронию судьбы,[1411]
1,/20181231/1548958617.html,на украине позавидовали стене на границе с крымом,"[1324, 114]"
2,/20181231/1548957394.html,в госдуме предложили отказаться от газа в жилы...,"[1353, 1353, 601, 1532, 1075]"
3,/20181231/1548954909.html,названы лучшие средства от похмелья,[1411]
4,/20181231/1548957120.html,новогоднее поздравление порошенко разозлило по...,"[1324, 114]"


Напишем датасет для поставки данных в нейросеть.


In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size,
                 max_title_len, max_classes,
                 snippet=None, max_snippet_len=None):

       self.vocab = {word: idx for word,idx in vocab.items() if idx < vocab_size}
       self.max_classes = max_classes
       self.y = self.target_ohe(target)
       self.X_title = self.create_text(title, max_title_len)

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = [ ]
        for sent in texts:
            # {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]

            # приводим все тексты к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.y[idx])


__[0.2 балла] Задание 3:__ Сейчас датасет умеет работать только с полем `title_clean`. Давайте сделаем этот датасет более многофукнциональным и добавим в него возможность добавить в обработку данных сниппет.

1. Внутри датасета `snippet` надо обработать точно также как и `title`.
2. Если `snippet=None`, датасет должен вернуть два объекта: `X_title, y`. В обратном случае датасет должен вернуть три объекта.

**Важно:** Весь код ниже работает сейчас без сниппета. Он не должен развалиться от того, что сниппет в нём нигде не указан.

Объявим датасеты, оставим в словаре 30 000 самых частотных слов. Будем смотреть на титулы максимальной длины 20.

In [ ]:
CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20

# словарь создаем по всей выборке
vocabulary = create_vocab(' '.join(df_ria.title_clean.values))

# объявляем датасеты
train_dataset = NewsDataset(df_train.target_tags.values, df_train.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
val_dataset = NewsDataset(df_val.target_tags.values, df_val.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
test_dataset = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4096, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## 1.4 Архитектуры

Соберём базовую архитектуру для обучения.

In [ ]:
from torch import nn
import torch.nn.functional as F

class SimpleClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, title):
        embedded = self.embedding(title)
        embedded = embedded.mean(dim=1)
        embedded = self.fc(embedded)
        return embedded

Соберём в `pytorch_lightning` модуль для обучения нейронки.

In [ ]:
!pip3 install pytorch_lightning
import pytorch_lightning as pl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 45.4 MB/s eta 0:00:00


In [ ]:
class TrainLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title):
        result = self.model(title)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

Обучим модель.

In [ ]:
EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_baseline_no_snippet = SimpleClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module = TrainLightningModule(model_baseline_no_snippet, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | SimpleClassifier | 3.5 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.906    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
# изучить, что происходит в форварде модели, чтобы выполнить задание по модификации

# model_baseline

# dataloader выдаёт два объекта (если со сниппетами, то три, получается):
# 1) 64 штуки инпута (каждый -- закодированный тайтл/сниппет в векторе размером 20,
# где каждое число -- порядковый номер в общем вокабуляре; т.е. мы обрезали все
# тексты по 20 -- такое число дефолтное, можно менять, и каждому слову, которое нашли
# в общем воукабуляре предварительно сделанном, присвоили соответствующий номер)
# 2) 64 штуки предсказаний длиной 1583 -- 1583 темы, и под каждой релевантной
# единичка

# model_baseline принимает на вход батч из 64 инпутов (дефолтный размер батча)
# и с каждым вектором из батча такого вида:

# tensor([ 180,    1, 1322, 7839,    1,    1,  399,    1, 1665,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0], dtype=torch.int32)

# делает
# 1:
# эмбеддинг 20x300 (300 -- это размерность эмбеддинга, которую мы выбрали)
# 20 -- это длина подаваемого вектора, собственно, его координаты в 20-мерном
# пространстве. Т.е. мы делаем из 20х1 вектора 20х300 эмбеддинг
# tensor([[-0.2356,  0.2516, -0.6296,  ..., -0.7635, -0.6776,  1.0371],
#         [-0.0994, -1.0835,  1.3833,  ..., -0.1399, -0.4887, -1.0935],
#         [ 0.6162,  0.1217,  0.3919,  ..., -0.7937, -0.0183,  0.4522],
#         ...,
#         [-1.4050,  1.3867, -0.9091,  ...,  0.5418,  1.3386,  1.2188],
#         [-1.4050,  1.3867, -0.9091,  ...,  0.5418,  1.3386,  1.2188],
#         [-1.4050,  1.3867, -0.9091,  ...,  0.5418,  1.3386,  1.2188]],
#        grad_fn=<EmbeddingBackward0>)

# 2: каждая 20-элементная строчка схлапывается в среднее, получаем
# 300-элементный вектор:
#
# tensor([ 1.1373e+00, -1.4899e+00,  4.3812e-01,  3.2340e-01,  5.6278e-01,
#         -1.7950e-01, -9.5360e-01, -4.2200e-01, -1.1328e+00,  5.3756e-01,
#         -2.5282e-01, -5.3846e-01, -1.2483e+00,  5.7790e-01, -1.0328e-01,
#          4.3460e-01, -3.6170e-01,  9.0517e-01, -1.2324e-02, -5.7085e-01,
#          6.6225e-02,  5.7611e-01, -3.8877e-01, -1.0584e+00,  7.9612e-01,
#         -1.3026e+00, -8.2512e-01,  1.0255e+00,  9.7224e-02, -5.0049e-02,
#         -4.3208e-02, -4.5853e-01, -1.3447e+00, -3.1119e-03, -5.9222e-01,
#          2.6519e-01, -7.7165e-01,  1.1178e+00,  1.4212e+00, -3.6295e-01,
#         -1.1778e-01,  1.0329e+00, -8.3781e-01,  1.6759e-01, -5.1524e-01,
#         -8.7431e-01,  3.9499e-02, -3.7634e-01,  1.5573e-01, -1.2211e-01,
#         -8.7480e-02, -1.4332e+00, -3.1440e-01, -2.7268e-01,  9.9552e-01,
#         -1.9598e-01,  8.1554e-01,  7.2380e-01, -3.4376e-01, -2.3896e-01,
#          3.2630e-01, -2.0081e-01,  1.9628e+00,  8.1074e-01,  1.4077e+00,
#         -4.3164e-01,  5.3228e-01, -3.4512e-01, -3.9873e-01, -7.0100e-02,
#         -8.3617e-01, -1.1534e+00,  5.3724e-01,  1.8051e+00, -3.0009e-02,
#          1.0670e+00, -3.9656e-01, -3.6855e-01, -8.3374e-01,  7.4913e-02,
#          8.8010e-01,  6.6062e-01,  2.1763e-01,  5.1113e-01, -9.2596e-01,
#         -4.2117e-02, -1.3417e-01, -8.3905e-02,  2.6022e-01,  1.3227e-01,
#         -9.5199e-02, -9.0340e-01, -1.0084e+00,  1.7364e-01,  1.4368e-01,
#          2.0006e-01,  2.2905e-01,  1.2303e+00,  1.9168e-01,  3.5130e-02,
#         -7.4677e-03, -5.8598e-02,  7.1716e-01,  2.3991e-01, -1.7515e-01,
#         -1.2583e+00,  6.2091e-02,  3.1998e-01,  4.0521e-01,  2.7597e-01,
#          5.8060e-01,  6.0249e-01,  4.6862e-01, -2.0767e-01,  3.4025e-01,
#          6.3098e-01, -3.7040e-01, -7.0652e-01, -3.3574e-01, -3.3856e-01,
#         -1.3833e-01,  4.0741e-01,  4.5969e-01, -4.1504e-01,  2.2828e-01,
#         -2.0863e-01,  4.8330e-01, -6.8731e-01, -2.7504e-01, -6.2352e-01,
#         -4.1425e-01, -5.1235e-01, -1.5478e-01,  3.8703e-01,  7.5114e-01,
#         -3.6104e-01,  4.7486e-01,  7.6954e-01,  1.3969e+00, -1.1183e+00,
#         -1.0510e+00,  9.5201e-02,  1.1355e+00, -1.9196e-02,  6.8978e-02,
#          2.5523e-01, -3.0851e-01,  6.9626e-01, -8.4270e-01, -1.0417e-01,
#         -1.6762e-01, -1.0786e+00, -1.0260e+00, -1.2672e+00, -1.2102e-01,
#         -1.5479e+00,  8.4852e-01,  6.4554e-01, -1.2387e+00, -7.1664e-01,
#         -3.9096e-01,  8.2944e-01, -3.4953e-01, -2.0601e-01, -4.7962e-01,
#          2.9351e-01,  8.3036e-01,  9.3719e-01,  5.0040e-01,  5.9409e-01,
#         -7.4980e-03,  6.8661e-02,  1.1442e+00,  1.1636e+00, -1.2316e-01,
#          8.4619e-01, -8.1114e-01, -1.4916e-01,  1.2677e-01,  7.8219e-02,
#         -1.1315e+00,  3.4951e-01,  7.9703e-01,  8.9013e-01,  5.4687e-01,
#          3.1717e-01, -1.8557e-01, -7.4992e-01, -2.4481e-01,  6.8497e-01,
#         -3.8922e-01, -4.1004e-02, -1.2229e-01,  3.9216e-01, -1.0006e+00,
#         -5.3687e-02, -1.6828e-01,  8.8313e-01,  8.4266e-01,  2.6864e-01,
#         -1.3652e+00,  4.2344e-01, -2.6810e-01,  5.1597e-01,  9.8241e-02,
#         -2.1326e-01, -4.2813e-01,  8.1501e-01,  2.4713e-01,  2.9194e-01,
#          3.2088e-01, -3.7858e-02, -5.1649e-01, -5.3387e-01,  6.0668e-04,
#         -8.7425e-02, -1.8361e-01,  1.4816e-01,  4.7039e-01,  1.2576e+00,
#          4.9153e-01,  1.1363e+00, -4.7182e-01,  3.0755e-01, -8.0170e-01,
#         -1.5314e-01, -1.1221e-01, -6.7959e-01,  1.1393e-01,  3.0501e-01,
#          3.7417e-01, -3.2760e-02, -1.9968e-01,  1.9328e-02,  4.6664e-01,
#         -5.1453e-02, -1.7595e-01, -4.1158e-01, -1.8288e-01,  6.7739e-01,
#          5.9020e-01,  2.0407e-01,  8.0509e-02, -1.3577e+00, -1.1676e+00,
#         -8.3822e-01, -1.7897e-03,  3.9706e-01,  4.4542e-01,  3.9257e-01,
#         -2.5296e-01,  1.6685e-02,  2.5559e-01,  5.7308e-01,  1.4395e+00,
#         -1.6368e-01,  4.3225e-01, -8.9749e-01,  4.3203e-01,  2.9687e-01,
#          2.5431e-01,  8.4404e-01, -1.9263e-01, -5.6193e-01, -2.4370e-01,
#          5.9500e-01,  5.3560e-01,  1.2159e-02, -7.3908e-01, -3.1619e-01,
#          3.9741e-01,  5.2120e-01, -5.2077e-02, -3.6625e-01, -7.8027e-01,
#          4.8538e-02,  3.8579e-01,  1.2962e-01,  1.0805e-01, -4.7309e-01,
#          8.0808e-01, -4.2618e-02,  1.1939e-01,  2.7005e-02,  3.9551e-01,
#          4.9094e-02, -1.0078e+00,  7.9784e-01, -7.4128e-02,  1.0520e+00,
#         -1.5635e+00,  1.5751e+00, -4.4504e-01, -5.9041e-01,  3.1311e-01,
#          1.9469e-01, -5.7621e-01, -3.3650e-01, -2.9265e-01, -5.6642e-02],
#        grad_fn=<SelectBackward0>)

# и из него уже линейным слоем делаем какое-то предсказание
# вектор из 1583 элементов (темы), по каждому из которых что-то типа вероятности
# или силы присутствия в оригинальном тексте

# tensor([-0.1907,  0.0350,  1.2044,  ...,  0.3400,  0.4569,  0.2984],
#       grad_fn=<SelectBackward0>)


train_dataloader = DataLoader(train_dataset, shuffle=False, batch_size=64, num_workers=4)


for elem in train_dataloader:
  title_batch = elem[0]
  break

vec = title_batch
pred = model_baseline_no_snippet(vec)
pred[0]


tensor([-0.0245, -0.1300, -0.5422,  ..., -0.0635, -0.2916, -0.0582],
       grad_fn=<SelectBackward0>)

__[0.2 балла] Задание 4:__  Модель выше работает только с титулом.

- Залоггируйте её обучение на [WandB](https://wandb.ai/).
- Соберите архитектуру, которая будет принимать на вход не только титул, но ещё и сниппет. В этой архитектуре должно происходить следующее:

1. Общий слой `nn.Embedding` применяется к сниппету и титулу параллельно.
2. Происходит усреднее по текстам.
3. Вектора конкатятся в один длины 600
4. Линейный слой делает классификацию

Обучите эту модель. Сравните траектории обучения на WandB. Прикрепите ссылку на дашборд либо скришот к тетрадке.

Даталоадеры придётся объявить заново с учётом сниппетов. Правда ли, что она бьёт на валидационной выборке модель, обученную только на титулах статей?

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size,
                 max_title_len, max_classes,
                 snippet=None, max_snippet_len=None):

       # target -- значения target_tags ([1024, 123])
       # title -- значения title_clean ('митрополит рязанский...')
       # vocab -- сделанный по title_clean вокабуляр
       # vocab_size и max_title_len и max_snippet len -- значения (число слов)

      #  snippet -- значения snippet_clean ('вашингтон 30 сен...')

       self.vocab = {word: idx for word,idx in vocab.items() if idx < vocab_size}
       self.max_classes = max_classes
       self.y = self.target_ohe(target)
       self.X_title = self.create_text(title, max_title_len)

       self.snippet_on = False

       if max_snippet_len != None:
          self.snippet_on = True
          self.X_snippet = self.create_text(snippet, max_snippet_len)



    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = [ ]
        for sent in texts:
            # {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]

            # приводим все тексты к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
      if self.snippet_on:
        return (self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx])
      else:
        return (self.X_title[idx, :], self.y[idx])


In [ ]:
class TrainLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title, snippet):

        result = self.model(title, snippet)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, snippet, target = train_batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, snippet, target = val_batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

In [ ]:
class TitleSnippetClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, title, snippet):
        embedded_title = self.embedding(title)
        embedded_title = embedded_title.mean(dim=1)

        embedded_snippet = self.embedding(snippet)
        embedded_snippet = embedded_snippet.mean(dim=1)

        concatted = torch.cat((embedded_title, embedded_snippet), dim=1)


        pred = self.fc(embedded_title)

        return pred

In [ ]:
CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20
MAX_SNIPPET_LEN = 50


# объявляем датасеты
train_dataset_snip = NewsDataset(df_train.target_tags.values, df_train.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM,
                            df_train.snippet_clean.values , MAX_SNIPPET_LEN)
val_dataset_snip = NewsDataset(df_val.target_tags.values, df_val.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM,
                          df_val.snippet_clean.values , MAX_SNIPPET_LEN)
test_dataset_snip = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM,
                           df_test.snippet_clean.values , MAX_SNIPPET_LEN)

train_dataloader = DataLoader(train_dataset_snip, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset_snip, shuffle=False, batch_size=4096, num_workers=4)

In [ ]:
EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_baseline = TitleSnippetClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module = TrainLightningModule(model_baseline, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | model     | TitleSnippetClassifier | 3.5 M  | train
1 | criterion | CrossEntropyLoss       | 0      | train
-------------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.906    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

## 1.5 Инференс и оценка качества моделей

Для каждой новости нам надо предсказывать несколько тэгов. То есть в нашем случае настоящее значение таргета это множество из тэгов $y_i = [tag1, tag2, tag3]$. Прогноз модели также множество из тэгов $\hat y_i = [tag1, tag4]$.

Будем считать метрики качества следующим образом (под $|A|$ имеется в виду мощность множества, то есть число элементов в нём):

$$
Precision = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|\hat{y}_i|}
$$

$$
Recall = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|y_i|}
$$

Также можно считать аналог Accuracy, но это не самая удачная идея, так как у нас в выборке огромное число нулей и эта метрика при любом разумном пороге для принятия решения будет очень высокой.

$$
Exact Match = \frac{1}{n} \cdot \frac{1}{k} \sum_{i = 1}^n \sum_{k=1}^K [y_{ij} = \hat{y}_{ij}]
$$

In [ ]:
def precision(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (y_pred == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def recall(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (target == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def exact_match(target, y_pred):
    return (1.*(y_pred == target)).mean().item()

Построим прогноз на тестовой выборке.

In [ ]:
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=test_dataset.__len__())

for title, target in test_dataloader:
    logit = model_baseline_no_snippet(title)
    pred_prob = F.softmax(logit, dim=1)

assert pred_prob.shape[0] == test_dataset.__len__()

NameError: name 'test_dataset' is not defined

Теперь выбирая различное значение порога, мы можем получать разные предсказания. Если взять очень большое значение порога, то метрики сильно просядут, так как во многих документах никакого прогноза не будет построено вообще.

In [ ]:
TRESHOLD = 0.01
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9955514669418335
Precision: 0.29185381531715393
Recall: 0.7612126469612122


In [ ]:
TRESHOLD = 0.05
y_pred = 1*(pred_prob > TRESHOLD)
print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983208775520325
Precision: 0.5195301175117493
Recall: 0.6177495718002319


In [ ]:
TRESHOLD = 0.9
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983181953430176
Precision: 0.005798910744488239
Recall: 0.0049522072076797485


Дальше мы будем строить довольно много прогнозов. Давайте напишем код для их строительства в виде функции. Обратите внимание, что на модели со снипетом она упадёт. Когда вы доберётесь до строительства прогнозов, функцию придётся немного модернизировать.

In [ ]:
def get_predict(model, dataset):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=dataset.__len__())

    for title, target in dataloader:
        logit = model(title)
        pred_prob = F.softmax(logit, dim=1)

    assert pred_prob.shape[0] == dataset.__len__()
    return pred_prob, target

In [ ]:
def get_predict_snippet(model, dataset):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=dataset.__len__())

    for title, snippet, target in dataloader:
        logit = model(title, snippet)
        pred_prob = F.softmax(logit, dim=1)

    assert pred_prob.shape[0] == dataset.__len__()
    return pred_prob, target

__[0.2 балла] Задание 5:__ Какая метрика для нас в этой задаче важнее? Точность или полнота? Почему?

__ваш ответ:__
полнота

- Напишите функцию, которая будет подбирать оптимальное значение порога, оптимизирующее выбранную вами метрику.
- Подберите значение порога на валидационной выборке.
- Сравните модель со сниппетами и без сниппетов, используя выбранную вами метрику при оптимальном значении порога на тестовой выборке.
- Какая из них оказалась лучше?

In [ ]:
# your code here
# (＠_＠)


def optimize_threshold(y_pred_, target_, f_metric):
  thresholds = np.arange(0.01, 1, 0.01)
  best_pair = {'threshold': None,
               'result': 0}

  pbar = tqdm(thresholds)
  for threshold in pbar:
    y_pred_t = 1*(y_pred_ > threshold)
    result = f_metric(target_, y_pred_t)
    pbar.set_description(f'Optimising {f_metric.__name__}, current best: {best_pair}')
    if result > best_pair['result']:
      best_pair['threshold'] = threshold
      best_pair['result'] = round(result, 2)

  return best_pair

# metrics = [precision, recall, exact_match]

# besties = {}
# for metric in metrics:
#   res = optimize_threshold(pred_prob, target, metric)
#   besties[metric.__name__] = res

# besties

In [ ]:
y_pred, target = get_predict_snippet(model_baseline, test_dataset_snip)

metrics = [precision, recall, exact_match]

besties = {}
for metric in metrics:
  res = optimize_threshold(pred_prob, target, metric)
  besties[metric.__name__] = res

besties

NameError: name 'get_predict_snippet' is not defined

__[0.2 балла] Задание 6:__  Постройте прогнозы для отложенной выборки, которая представляет из себя пересечение сайта РИА-новостей и ВКонтакте. Проседает ли на ней качество модели? Насколько сильно?

In [ ]:
df_oob.head()


,href,title_clean,target_tags
0,/20181231/1548960744.html,митрополит рязанский раскритиковал иронию судьбы,[188]
1,/20181231/1548958617.html,на украине позавидовали стене на границе с крымом,"[970, 813]"
2,/20181231/1548957394.html,в госдуме предложили отказаться от газа в жилы...,"[1513, 1513, 1208, 767, 1486]"
3,/20181231/1548954909.html,названы лучшие средства от похмелья,[188]
4,/20181231/1548957120.html,новогоднее поздравление порошенко разозлило по...,"[970, 813]"


In [ ]:
oob_dataset = NewsDataset(df_oob.target_tags.values, df_oob.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM)

# y_pred, target = get_predict(model_baseline_no_snippet, oob_dataset)


# metrics = [precision, recall, exact_match]

# besties = {}
# for metric in metrics:
#   res = optimize_threshold(y_pred, target, metric)
#   besties[metric.__name__] = res

# besties

In [ ]:
# ПРЕДВАРИТЕЛЬНЫЕ ШТУКИ ДЛЯ БОНУСНОГО ЗАДАНИЯ


train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4096, num_workers=4)

oob_dataset = NewsDataset(df_oob.target_tags.values, df_oob.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM)





class TrainLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title):
        result = self.model(title)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

## 1.6 Бонусное задание

Давайте модернизируем наши архитектуры настолько, насколько это возможно.

__[0.5 балла]__ Попробуйте собрать более большую архитектуру. Например, сразу после слоя эмбеддингов вы можете попробовать добавить свёрточные слои (`Conv1D` свёртки). Поиграйте с оптимизатором и тп.

Опишите результаты своих экспериментов ниже. Расскажите, что конкретно вы делали и удалось ли вам улучшить качество модели. Все траектории обучения залоггируйте на WandB.   

__Ваш лог экспериментов:__

*без сниппетов: precision / recall*
- baseline: 0.56 0.78
- '+ nn.Conv1d(20, 10, 3, padding=1): 0.57 0.77
- '+ nn.Conv1d(20, 10, 3, padding=1) ReLU: 0.54 0.77
- '+ nn.Conv1d(20, 10, 3, padding=1) fc1 ReLU fc2: 0.56 0.77
- fc1 ReLU fc2 ReLU fc3: 0.5 0.75



-

In [ ]:
class EnhancedClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # self.conv1 = nn.Conv1d(20, 10, 3, padding=1)
        self.fc1 = nn.Linear(300, 150)
        self.fc2 = nn.Linear(150, 50)
        self.fc3 = nn.Linear(50, output_dim)
        self.relu = nn.ReLU()

    def forward(self, title):
        embedded = self.embedding(title)
        # print(embedded.size())
        # embedded = self.conv1(embedded)
        # print(embedded.size())
        embedded = embedded.mean(dim=1)
        # print(embedded.size())
        embedded = self.fc1(embedded)
        embedded = self.relu(embedded)
        embedded = self.fc2(embedded)
        embedded = self.relu(embedded)
        embedded = self.fc3(embedded)

        return embedded


EMBEDDING_DIM = EMBEDDING_DIM
EPOCHS = 5
LR = 1e-3

enhanced_model = EnhancedClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module = TrainLightningModule(enhanced_model, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)

y_pred, target = get_predict(enhanced_model, oob_dataset)


metrics = [precision, recall, exact_match]

besties = {}
for metric in metrics:
  res = optimize_threshold(y_pred, target, metric)
  besties[metric.__name__] = res

besties

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | EnhancedClassifier | 3.1 M  | train
1 | criterion | CrossEntropyLoss   | 0      | train
---------------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.534    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

{'precision': {'threshold': 0.11, 'result': 0.5},
 'recall': {'threshold': 0.01, 'result': 0.75},
 'exact_match': {'threshold': 0.02, 'result': 1.0}}

__[0.5]__ Скачайте с сайта [Rusvectores](https://rusvectores.org/ru/models/) любые новостные word2vec эмбединги. Возьмите из модели эмбеддинги для всех слов, которые встречаются вв вашем словаре и добавьте их в модель первым слоем. Заморозьте этот слой и не обновляйте в нём веса. Если у вас в словаре есть слово, но его нет среди предобученных эмбеддингов, замените его на токен `#UNKN`.

__Ваш лог экспериментов:__

результат на эмбеддингах с http://vectors.nlpl.eu/repository/20/184.zip


{'precision': {'threshold': 0.06999999999999999, 'result': 0.25},
 'recall': {'threshold': 0.01, 'result': 0.59},
 'exact_match': {'threshold': 0.03, 'result': 1.0}}

In [ ]:
# качаем датасет
!wget http://vectors.nlpl.eu/repository/20/184.zip

!unzip /content/184.zip

--2024-11-17 19:27:18--  http://vectors.nlpl.eu/repository/20/184.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.200, 2001:700:112::200
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 640196018 (611M) [application/zip]
Saving to: ‘184.zip’

184.zip             100%[===================>] 610.54M  17.6MB/s    in 36s     

2024-11-17 19:27:54 (17.0 MB/s) - ‘184.zip’ saved [640196018/640196018]

Archive:  /content/184.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [ ]:
# вытаскиваем вектора, обрабатываем, кладём в словарь

with open("/content/model.txt", "r") as ins:
    array = []
    for line in ins:
        array.append(line)

array = array[1:]

def process_arr(arr):
  arr_list = arr.split(sep=' ')
  word = arr_list[0]
  word = re.sub(r'_.+', '', word)
  vec = list(map(float, arr_list[1:]))

  return word, vec


w2v = {}

for line in tqdm(array):
  word, vec = process_arr(line)
  w2v[word] = vec


  0%|          | 0/249318 [00:00<?, ?it/s]

In [ ]:
voc = list(vocabulary.keys())[2:]

words = list(w2v.keys())

In [ ]:
# Готовим словарь с эмбеддингами, будем их подавать на вход модели
embeddings = {}
for word in tqdm(words):
  if word in voc:
    embeddings[word] = w2v[word]



  0%|          | 0/215875 [00:00<?, ?it/s]

In [ ]:
embeds = {}
for word in tqdm(embeddings):
  fig = vocabulary[word]
  embed[fig] = embeddings[word]



  0%|          | 0/29259 [00:00<?, ?it/s]

In [ ]:
# надо отсортировать словарь

sorted_embeds = dict(sorted(embeds.items()))

# добавить 0 -- pad и 1 -- unkn, взяв какие-нибудь вектора (сгенерировать?)
pad = torch.rand(300).tolist()
unkn = torch.rand(300).tolist()

sorted_embeds[0] = pad
sorted_embeds[1] = unkn
sorted_embeds = dict(sorted(sorted_embeds.items()))

# и заполнить пробелы между числами,
# идущими не подряд, нулями

last_elem = len(vocabulary)

for n in tqdm(range(last_elem)):
  try:
    sorted_embeds[n]
  except:
    sorted_embeds[n] = unkn


  0%|          | 0/112030 [00:00<?, ?it/s]

In [ ]:
import pickle

# with open('sorted_embeds.pickle', 'wb') as f:
  # pickle.dump(sorted_embeds, f)

with open('sorted_embeds.pickle', 'rb') as f:
  sorted_embeds = pickle.load(f)


In [ ]:
# делаю из sorted_embeds pretrained weights для первого слоя

weights = list(sorted_embeds.values())
pretrained = torch.tensor(weights)

len(pretrained)


112030

In [ ]:
# берём из модели эмбеддинги всех слов, которые есть в словаре,\
# добавляем их в модель первым слоем

class ClassifierWithEmbeddings(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(pretrained)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, title):
        embedded = self.embedding(title)
        # print('заембеддиили')
        embedded = embedded.mean(dim=1)
        embedded = self.fc(embedded)
        return embedded


train_dataloader = DataLoader(train_dataset, shuffle=False, batch_size=64, num_workers=4)

EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_pretrained = ClassifierWithEmbeddings(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module = TrainLightningModule(model_pretrained, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                     | Params | Mode 
---------------------------------------------------------------
0 | model     | ClassifierWithEmbeddings | 34.1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
y_pred, target = get_predict(model_pretrained, oob_dataset)


metrics = [precision, recall, exact_match]

besties = {}
for metric in metrics:
  res = optimize_threshold(y_pred, target, metric)
  besties[metric.__name__] = res

besties

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

{'precision': {'threshold': 0.06999999999999999, 'result': 0.25},
 'recall': {'threshold': 0.01, 'result': 0.59},
 'exact_match': {'threshold': 0.03, 'result': 1.0}}

__[1 балл]__ Зафайнтьюньте трансформер для решения задачи с помощью библиотеки `hugging face`. Выбор предобученной модели кратко обоснуйте.

__Ваш лог экспериментов:__

-
-
-

In [ ]:
!pip install transformers sentencepiece --quiet
import transformers

In [ ]:
# выбираем трансформер-модель, которая пойдёт для предсказания категории
#  (классификация)
# импортируем, разбираемся с интерфейсом,

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', max_length=20)
# model = BertModel.from_pretrained("bert-base-multilingual-cased")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# нужно написать даталоадер с векторайзером от берта

In [ ]:
class NewsDatasetBert(Dataset):

    def __init__(self, target, titles, max_classes, tokenizer, max_title_len):

       self.max_classes = max_classes
       self.y = self.target_ohe(target)
       self.X_title = titles
       self.max_length = max_title_len


    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y


    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        tokens = self.X_title[idx]
        tokens = tokens.split(' ')
        if len(tokens) > self.max_length:
          tokens = tokens[:self.max_length]
        while len(tokens) < self.max_length:
          tokens.append('[UNK]')
        tokens = ' '.join(tokens)
        res = tokenizer(tokens, return_tensors='pt',
                        max_length=self.max_length,
                        padding='max_length',
                        truncation=True, is_split_into_words=True,
                        add_special_tokens = True)['input_ids'][0]
        # print(f'{res.size() = }')


        # tokens = tokenizer.batch_encode_plus(tokens, max_length=self.max_length, return_tensors='pt', padding='max_length',
                                    #  truncation=True, is_split_into_words=True, add_special_tokens = True)['input_ids']
        # tokens = tokenizer.batch_encode_plus(tokens, max_length=10, padding='max_length',
                                    #  truncation=True, is_split_into_words=True)['input_ids']

        return res, self.y[idx].type(torch.LongTensor)

CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20


# объявляем датасеты
train_dataset = NewsDatasetBert(df_train.target_tags.values, df_train.title_clean.values, CLASSES_NUM, tokenizer, MAX_TITLE_LEN)
val_dataset = NewsDatasetBert(df_val.target_tags.values, df_val.title_clean.values, CLASSES_NUM, tokenizer, MAX_TITLE_LEN)
test_dataset = NewsDatasetBert(df_test.target_tags.values, df_test.title_clean.values, CLASSES_NUM, tokenizer, MAX_TITLE_LEN)

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, output_dim):
        super().__init__()
        model = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.bert_part = nn.Sequential(*list(model.children())[0:-1])
        for param in self.bert_part.parameters():
          param.requires_grad = False

        self.fc = nn.Linear(768, output_dim)
        self.softmax = nn.Softmax()

    def forward(self, title):
        embedded = self.bert_part(title)['last_hidden_state']
        # print(f'{embedded.size() = }')
        embedded = self.fc(embedded)
        # print(f'{embedded = }')

        return self.softmax(embedded)


In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=64, num_workers=4)

EPOCHS = 5
LR = 1e-3

model_pretrained = BertClassifier(CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module = TrainLightningModule(model_pretrained, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | BertClassifier   | 178 M  | train
1 | criterion

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Сравните все обученные модели между собой на тестовой выборке.

In [ ]:
y_pred, target = get_predict(model_pretrained, oob_dataset)


metrics = [precision, recall, exact_match]

besties = {}
for metric in metrics:
  res = optimize_threshold(y_pred, target, metric)
  besties[metric.__name__] = res

besties

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


  0%|          | 0/99 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (20) must match the size of tensor b (1133) at non-singleton dimension 1

## Часть 2: предсказание категорий (0.3 балла)

Возьмите датасет `df_vk` и для всех новостей из него предскажите категории с помощью лучшей, получившейся у вас модели.

In [ ]:
CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20

df_vk['target_tags'] = [[0]]*df_vk.shape[0]


model = model_baseline_no_snippet



vk_dataset = NewsDataset(df_vk['target_tags'].values, df_vk['title_clean'].values,
                         vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM)



In [ ]:
y_pred, targets = get_predict(model, vk_dataset)

y_pred


tensor([[1.9397e-04, 3.9660e-06, 1.2938e-06,  ..., 1.9939e-04, 2.4723e-06,
         3.2529e-06],
        [2.4425e-06, 7.1623e-04, 6.8626e-06,  ..., 7.4122e-06, 4.4085e-06,
         1.7314e-05],
        [8.9101e-04, 7.7794e-04, 9.0108e-05,  ..., 2.1636e-04, 1.1917e-05,
         5.6000e-06],
        ...,
        [1.8181e-05, 1.2777e-04, 7.8441e-04,  ..., 1.6716e-05, 1.7068e-05,
         2.6493e-05],
        [5.2561e-05, 3.5549e-05, 6.4822e-06,  ..., 7.9463e-05, 9.1823e-07,
         3.8478e-07],
        [5.5403e-05, 4.6823e-04, 5.5234e-05,  ..., 1.5394e-04, 1.2215e-04,
         1.7578e-04]], grad_fn=<SoftmaxBackward0>)

In [ ]:
categories_pred = y_pred.argmax(dim=1)

categories_vec = []

for idx in categories_pred:
  categories_vec.append(idx2tag[idx.item()])

In [ ]:
df_vk['category_predicted'] = categories_vec

In [ ]:
df_vk.loc[:, ['title', 'category_predicted']]
# прикольно получилось

,title,category_predicted
0,"В ДНР заявили о задержании диверсантов, причас...",в мире
1,"Житель Урала ""заминировал"" ТЦ из-за снятия со...",происшествия
2,"В Черном море нашли ""потерянный флот Гитлера""",россия
3,В США освободили задержанную российскую актрис...,сша
4,Толкнувший Скабееву депутат Рады заявил о гроз...,в мире
...,...,...
19923,"Политологи назвали губернаторов, которым грози...",политика
19924,"Захарова рассказала, какую книгу подписывал Ла...",россия
19925,Современные люди появились более 300 тысяч лет...,риа наука
19926,В США ребенок устроил стрельбу,в мире


In [ ]:
import pickle

with open('simple_prediction.pickle', 'wb') as f:
  pickle.dump(y_pred, f)

На всякий случай сохраните табличку с получившимися у вас предсказаниями. Мало ли, вы не доделаете последнее задание, а потом захотите вернуться к нему. Не прогонять же обучение нейросети и инференс по второму кругу...

## Часть 3: сентимент-классификатор (0.5 балла)

В этой части тетрадки нам предстоит прогнать все комментарии из ВК через сентимент-классификатор. Мы будем делать это с помощью библиотеки HuggingFace. В ней есть удобная [функциональность pipline,](https://huggingface.co/docs/transformers/pipeline_tutorial) чтобы прогонять на своих данных уже обученные модели. 🤗🤗🤗

In [7]:
df_comments.head()

,id,post_id,datetime,text,likes
0,24006366.0,24006362.0,2019-02-01 23:14:14,ЧВК Вагнера?,5.0
1,24006370.0,24006362.0,2019-02-01 23:15:23,"[id4710641|Евгений], выздоравливай.",3.0
2,24006371.0,24006362.0,2019-02-01 23:16:21,"[id442655034|Андрей], искренне желаю этого все...",4.0
3,24006374.0,24006362.0,2019-02-01 23:16:38,Опять про Украину новости?,1.0
4,24006375.0,24006362.0,2019-02-01 23:16:40,Че такое ДНР?,2.0


Для классификации комментариев на позитивные, негативные и нейтральные будем использовать [модель, о которой можно почитать по ссылке](https://huggingface.co/seara/rubert-tiny2-russian-sentiment)

Ответьте на следующие вопросы:

1. Как зовут автора модели?
- *Сергей Сметанин*
2. Как выглядит архитектура модели? Насколько модель большая?
- *a small Russian BERT-based encoder with high-quality sentence embeddings*
- "*Это доработанная версия rubert-tiny: я расширил словарь модели c 30К до 80К токенов, увеличил максимальную длину текста с 512 до 2048 токенов, и дообучил модель на комбинации задач masked language modelling, natural language inference, и аппроксимации эмбеддингов LaBSE.*"
- *архитектура в аутпуте ниже*
- *число параметров 29 194 707*
3. На каких данных обучена модель? Адекватно ли использовать её для классификации комментариев ВК?
- This model was trained on the union of the following datasets:
-- Kaggle Russian News Dataset
-- Linis Crowd 2015
-- Linis Crowd 2016
-- RuReviews
-- RuSentiment

In [12]:
from torchsummary import summary

mdl = pipe.model

print(mdl)

sum(p.numel() for p in mdl.parameters())

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

29194707

Установим библиотеку. 🤗🤗🤗

In [9]:
!pip3 install transformers

In [56]:
from transformers import pipeline
pipe = pipeline(model="seara/rubert-tiny2-russian-sentiment", device='cuda', truncation=True)

Разберитесь как можно прогнать модель на корпусе комментариев и сделайте это. Да, с помощью pipeline можно запустить довольно сложные модели, обученные другими людьми в пару строчек. При объявлении модели не забудьте положить её на нужный `device` 🤗🤗🤗

In [57]:
dataset = df_comments.text.values.tolist()

verdicts = []
scores = []


# проверяем, есть ли пикл, если нет, то новый, если есть, то старый

import os, pickle
from google.colab import files

if os.path.exists('preds.pickle'):
    print('Загружаю файл с прогрессом...')
    with open('preds.pickle', 'rb') as f:
      preds = pickle.load(f)
else:
    print('Создаю новый файл для предсказаний')
    preds = {'n_to_start_with': 0,
            'verdicts': [],
            'scores': []}


i = preds['n_to_start_with']


batchsize = 128
pbar = tqdm(range(i, len(dataset), batchsize))

for n in pbar:
    pbar.set_description(f'Processing batch {n / batchsize :.0f} out of {len(dataset)/batchsize :.0f} (str {n})')
    batch = dataset[n:n+batchsize]
    pred = pipe(batch)
    try:
      for elem in pred:
        preds['verdicts'].append(elem['label'])
        preds['scores'].append(elem['score'])
      preds['n_to_start_with'] = n+batchsize
    except:
      raise Exception('Обеспечение атомарности')

    if n != 0 and n % (batchsize*100) == 0:
        with open('preds.pickle', 'wb') as f:
            pickle.dump(preds, f)
            print(f'Made dump as of {n / batchsize :.0f} batch')
            if n % (batchsize*5000) == 0:
              files.download("preds.pickle")


Загружаю файл с прогрессом...


  0%|          | 0/20211 [00:00<?, ?it/s]

Made dump as of 300 batch
Made dump as of 400 batch
Made dump as of 500 batch
Made dump as of 600 batch
Made dump as of 700 batch
Made dump as of 800 batch
Made dump as of 900 batch
Made dump as of 1000 batch
Made dump as of 1100 batch
Made dump as of 1200 batch
Made dump as of 1300 batch
Made dump as of 1400 batch
Made dump as of 1500 batch
Made dump as of 1600 batch
Made dump as of 1700 batch
Made dump as of 1800 batch
Made dump as of 1900 batch
Made dump as of 2000 batch
Made dump as of 2100 batch
Made dump as of 2200 batch
Made dump as of 2300 batch
Made dump as of 2400 batch
Made dump as of 2500 batch
Made dump as of 2600 batch
Made dump as of 2700 batch
Made dump as of 2800 batch
Made dump as of 2900 batch
Made dump as of 3000 batch


KeyboardInterrupt: 

Цикл для сентимент-анализа комментариев может работать довольно долго. Я крайне рекомендую вам переодически сохранять к себе на компьютер промежуточные результаты. Итоговый результат я рекомендую записать отдельным столбиком в таблицу с комментариями, а затем сохранить полученные результаты. 🤗🤗🤗

In [ ]:
df_vk['sentiment_verdict'] = ...
df_vk['sentiment_score'] = ...

# <ваш код для сохранения данных>

# your code here
# ¯\_(ツ)_/¯

# P.S. Код для скоринга комментариев работает довольно долго. Первый, кто проскорит всю базу, может поделиться ей в общем чате с однокурсниками
# Если вы используете эту базу, просто продемонстрируйте при решении этого пункта, что ваш код работает

## Часть 4: аналитика (1 балл + 0.2 бонусных)

Мы с вами огромные молодцы. Мы обучили модель для категоризации новостей, построили с её помощью прогнозы. Мы проскорили комментарии на их сентимент-окрас. Теперь давайте проанализируем новости. Описывайте полученные результаты таким образом, чтобы не получить уголовку на 5 лет за дискредитацию чего-нибудь или оскорбление чувств кого-нибудь 💜

__[0.2 балла]__ Какая доля комментариев позитивная? Какая доля комментариев негативная? Выведите 10 самых позитивных комментариев.

Выведите 10 самых негативных комментариев, поугарайте с них. Удалите их вывод из тетрадки. Никто не должен их видеть, это должно остаться только между нами. Поззитивные не удаляйте. Они пусть останутся.

In [ ]:
# your code here
# ʕ•ᴥ•ʔ

__[0.2 балла]__ Для каждой новости из датасета посчитайте количество негативных и позитивных комментариев под ней. Сохраните эти количества в виде новых колонок.

In [ ]:
# your code here
# [✖‿✖]

__[0.2 балл]__ Правда ли, что новости с большим количеством лайков получают больше негативных комментариев? А позитивных? Правда ли, что чем больше лайков, тем под новостью больше комментариев?

Постройте визуализацию, которая могла бы это проиллюстрировать.

In [ ]:
# your code here
# ┌(ಠ_ಠ)┘

__[0.2]__  Давайте построим по сентимент-окрасу комментариев топ позитивных новостей.

- Если под новостью оставлено 100 комментариев, из которых 80 позитивные, мы можем быть уверены в том, что новость была хорошо воспринята людьми.
- Если под новостью всего один комментарий и он оказался позитивным, то у нас 100% позитивных комментариев, но это вовсе не означает, что новость была воспринята хорошо.

Получается, что сортировать по доле позитивных комментариев нельзя. Давайте поступим умнее. Доля позитивных комментариев под постом -- это случайная величина. Её довольно часто моделируют с помощью бета-распределения. Если это случайная величина, мы можем построить для неё предиктивный интервал. Пусть $u$ - число позитивных комментариев, а $d$ - нейтральных и негативных.  Пусть

$$
a = 1 + u \qquad b = 1 + d.
$$

Тогда нижняя граница $95\%$ предиктивного интервала для доли будет вычисляться по такой формуле:

$$
\frac{a}{a + b} - 1.65 \cdot \sqrt{\frac{a \cdot b}{(a + b)^2 \cdot (a + b + 1)}}
$$

Если под новостью был всего один комментарий, у такой случайной величины будет высокая дисперсия. Это означаen, что штука, которую мы вычитаем из доли, окажется высокой. Левая граница интервала окажется маленькой и мы не поднимем комментарий в нашем топе наверх. Фактически мы делаем сортировку по квантилю уровня $0.05$ вместо среднего.

От вас требуется вбить эту формулу, сделать сортировку и вывести на экран топ позитивных новостей. Подробнее про то, откуда берётся эта формула можно почитать [в этой книге.](https://disk.yandex.ru/i/Ctd08bTwC9eI3g) Ищите 4 главу, страницу 140.

In [ ]:
# your code here
# ฅ^•ﻌ•^ฅ

Построили? Срочно пришлите свою любимую позитивную  новость в общий чат!!!

Топ негативных новостей строить не будем. Вокруг итак слишком много негатива 😻😻😻

__[0.2]__ Какие категории новостей оказались самыми позитивными? Придумайте способ найти такие категории и опишите его тут.

__Ответ:__

In [ ]:
# your code here
# ٩(⁎❛ᴗ❛⁎)۶

__[0.2 бонусных]__ Проанализируйте, как температура комментария (вероятность того, что он негативный) зависит от длины трэда (число комментариев под новостью)? Значима ли эта взаимосвязь? Если вам для проверки этого хочется построить линейную регрессию, не сдерживайтесь.

In [ ]:
# your code here
# ヾ(๑╹◡╹)ﾉ